In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [223]:
df=pd.read_csv("superstore.csv")
tempdf=df

Removing Miscellaneous(Unwanted) columns:

In [224]:
print(tempdf.columns)
tempdf.pop("Unnamed: 0")
tempdf.pop("记录数")
tempdf.pop("Row.ID")

Index(['Unnamed: 0', 'Category', 'City', 'Country', 'Customer.ID',
       'Customer.Name', 'Discount', 'Market', '记录数', 'Order.Date', 'Order.ID',
       'Order.Priority', 'Product.ID', 'Product.Name', 'Profit', 'Quantity',
       'Region', 'Row.ID', 'Sales', 'Segment', 'Ship.Date', 'Ship.Mode',
       'Shipping.Cost', 'State', 'Sub.Category', 'Year', 'Market2', 'weeknum'],
      dtype='object')


0        36624.0
1        37033.0
2        31468.0
3        31469.0
4        32440.0
          ...   
51285    33646.0
51286    33645.0
51287    32321.0
51288    35917.0
51289    37371.0
Name: Row.ID, Length: 51290, dtype: float64

Rearraging the columns for convenience

In [225]:
columns = [
    # Order Details
    "Order.ID", "Order.Date", "weeknum", "Year", "Ship.Date", "Order.Priority", "Ship.Mode",    
    # Customer Details
    "Customer.ID", "Customer.Name", "Segment",    
    # Location Details
    "City", "State", "Region", "Country",   
    # Product Details
    "Product.ID", "Product.Name", "Category", "Sub.Category",    
    # Sales and Financials
    "Quantity", "Sales", "Discount", "Profit",    
    # Market Details
    "Market", "Market2"
]
tempdf=tempdf[columns]

Cleaning the dataset:

In [226]:
for i in tempdf.columns:
    if(tempdf[i].dtypes=='object'):
        tempdf[i]=tempdf[i].astype("string")
tempdf['Order.Date']=pd.to_datetime(tempdf['Order.Date'])
tempdf['Ship.Date']=pd.to_datetime(tempdf['Ship.Date'])
tempdf['Year']=pd.to_numeric(tempdf['Year'])

#setting the dataframe index to show the row number
tempdf=tempdf.sort_values(by='Order.Date', ascending=False)

tempdf.reset_index()
tempdf.set_index(np.arange(0,len(tempdf)), inplace=True)
print(tempdf)

              Order.ID Order.Date  weeknum    Year  Ship.Date Order.Priority  \
0        ID-2014-54442 2014-12-31       53  2014.0 2015-01-04         Medium   
1         MO-2014-2560 2014-12-31       53  2014.0 2015-01-05         Medium   
2       MX-2014-110527 2014-12-31       53  2014.0 2015-01-02         Medium   
3       MX-2014-116267 2014-12-31       53  2014.0 2015-01-03           High   
4      IT-2014-3715679 2014-12-31       53  2014.0 2015-01-05           High   
...                ...        ...      ...     ...        ...            ...   
51285   CA-2011-131541        NaT       31  2011.0 2011-07-28           High   
51286   CA-2013-120250        NaT       38  2013.0 2013-09-23         Medium   
51287   CA-2014-131156        NaT       14  2014.0 2014-04-08         Medium   
51288   CA-2014-111262        NaT       44  2014.0 2014-11-02           High   
51289   US-2014-104661        NaT        3  2014.0 2014-01-19           High   

            Ship.Mode Customer.ID      

C:\Users\mooha\AppData\Local\Temp\ipykernel_26868\4085397441.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf[i]=tempdf[i].astype("string")
C:\Users\mooha\AppData\Local\Temp\ipykernel_26868\4085397441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf['Order.Date']=pd.to_datetime(tempdf['Order.Date'])
C:\Users\mooha\AppData\Local\Temp\ipykernel_26868\4085397441.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [227]:
print(len(tempdf))
tempdf = tempdf[tempdf['Ship.Date'] >= tempdf['Order.Date']]
print(df.duplicated().sum())
print(len(tempdf))

51290
0
50715


In [228]:
#filling up the Market and Market2 columns
M1=tempdf["Market"].dropna().unique().tolist()
M2=tempdf["Market2"].dropna().unique().tolist()
M2.append("North America")
#print(M1,"\n", M2)

for i in range(len(M1)): 
    tempdf.loc[tempdf["Market"].isna() & (tempdf["Market2"]==M2[i]), "Market"] = M1[i]
    tempdf.loc[tempdf["Market2"].isna() & (tempdf["Market"]==M1[i]), "Market2"] = M2[i]

Cleaning Location related information

In [229]:
c=pd.read_csv("cities.csv")
city_state_country=c.loc[:,["name", "state_name", "country_name"]]

In [230]:
# Creating a mapping dictionary 
state_to_country = dict(zip(city_state_country['state_name'], city_state_country['country_name']))
city_to_country = dict(zip(city_state_country['name'], city_state_country['country_name']))
city_to_state = dict(zip(city_state_country['state_name'], city_state_country['name']))

# mapping using np.where
tempdf['Country'] = np.where(tempdf['Country'].isna(), tempdf['State'].map(state_to_country), tempdf['Country'])
tempdf['Country'] = np.where(tempdf['Country'].isna(), tempdf['City'].map(city_to_country), tempdf['Country'])

tempdf['State'] = np.where(tempdf['State'].isna(), tempdf['City'].map(city_to_state), tempdf['State'])

print(len(tempdf[tempdf["Country"].isna()]))
print(len(tempdf[tempdf["State"].isna()]))
print(len(tempdf[tempdf["City"].isna()]))

26
348
244


In [231]:
tempdf.isnull().sum() 
tempdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50715 entries, 0 to 51084
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order.ID        50414 non-null  string        
 1   Order.Date      50715 non-null  datetime64[ns]
 2   weeknum         50715 non-null  int64         
 3   Year            50605 non-null  float64       
 4   Ship.Date       50715 non-null  datetime64[ns]
 5   Order.Priority  50582 non-null  string        
 6   Ship.Mode       50252 non-null  string        
 7   Customer.ID     50491 non-null  string        
 8   Customer.Name   50292 non-null  string        
 9   Segment         50291 non-null  string        
 10  City            50471 non-null  string        
 11  State           50367 non-null  object        
 12  Region          50305 non-null  string        
 13  Country         50689 non-null  object        
 14  Product.ID      50430 non-null  string        
 15  Product

In [232]:
freq_table = pd.crosstab(tempdf['Year'], tempdf['Segment'], margins=True, dropna=True, normalize=True) 
round(freq_table*100, 2)
#(freq_table/len(tempdf.dropna()))*100

Segment,Consumer,Corporate,Home Office,All
Year,,,,
2011.0,9.16,5.35,3.04,17.55
2012.0,11.28,6.37,3.72,21.37
2013.0,13.85,8.18,4.86,26.89
2014.0,17.43,10.15,6.60,34.18
All,51.72,30.06,18.22,100.00


In [233]:
#plt.bar(tempdf['Year'], tempdf['Sales'])

In [234]:
avgdate=tempdf['Ship.Date']-tempdf['Order.Date']
tempdf['avgdate']=avgdate